In [1]:
import requests
import csv
import re
import pandas as pd
import time
from bs4 import BeautifulSoup
import os
from numpy import arange
import string

In [2]:
inexp_page1 = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=western%20sydney&l=g%3A151.15106940214844%2C-33.67728783357405%2C150.7418286794922%2C-34.01944832216618&attrs=RestaurantsPriceRange2.1'

In [3]:
mod_page1 = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=western%20sydney&l=g%3A151.15106940214844%2C-33.67728783357405%2C150.7418286794922%2C-34.01944832216618&attrs=RestaurantsPriceRange2.2&ed_attrs=RestaurantsPriceRange2.1%2CRestaurantsPriceRange2.2'

In [4]:
pricey_page1 = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=western%20sydney&l=g%3A151.15106940214844%2C-33.67728783357405%2C150.7418286794922%2C-34.01944832216618&attrs=RestaurantsPriceRange2.3&ed_attrs=RestaurantsPriceRange2.1'

***Pages of reviews for each resturant*** <br>
1) We can use all characters between "datePublished.....and... /iv>" to capture the actual texts of reviews <br>
2) We can also use "ratingValue" content=" as a marker for star rating <br>
3) we will have to use the total number of reviews (found in plain html on the first page of all resturants) to calculate how many pages of reviews (assuming 20 reviews per page) 

***Pages of resturants for each price point***<br>
1) We can calculate total pages by using the total number of resturants (found in plain html at the top of each price point page) and then we assume 30 resturants per page <br>
2) We will have to also scrape all resturant names first and convert to all small case with dashes put between each word and punctuation marks removed in order to construct URLs for individual resturants for pre-review scraping <br>
3) We can use "ranking"....."name"......"neighbohoods" as markers for each resturant name we have to scrape and process

***Scrapper Pseudo Code***<br>
1- Calculate the number of pages of resturants each price point has <br>
2- Extract and process all resturant names for each price point <br>
3- Calculate the number of pages of reviews each resturant has <br>
4- Use numbers from above to establish dataframes and loops <br>
5- Finally scrape all reviews and star ratings into 4 seperate dataframes, one for each price point </br>

In [5]:
#Put price point pages in list
price_points = list()
price_points.append(inexp_page1)
price_points.append(mod_page1)
price_points.append(pricey_page1)

In [6]:
#list for resturants in each price point
pp_rest = list()

In [7]:
#get number of pages of resturants for each price point
for price_point in price_points:
    response = requests.get(price_point)
    content = response.content
    pp_rest.append(re.findall('Showing 1-30 of....',str(content)))

In [8]:
pp_rest

[['Showing 1-30 of 446'], ['Showing 1-30 of 820'], ['Showing 1-30 of 107']]

In [9]:
#dictionary for pages
price_rests = dict()
#extract last 3 numbers
pp_rest[0] = str(pp_rest[0])
rests = re.findall('[0-9]{3}',str(pp_rest[0]))
rests = str(rests)
rests = rests[2:5]
price_rests['inexp'] = int(rests)
#---------------------------------------
pp_rest[1] = str(pp_rest[1])
rests = re.findall('[0-9]{3}',str(pp_rest[1]))
rests = str(rests)
rests = rests[2:5]
price_rests['mod'] = int(rests)
#---------------------------------------
pp_rest[2] = str(pp_rest[2])
rests = re.findall('[0-9]{3}',str(pp_rest[2]))
rests = str(rests)
rests = rests[2:5]
price_rests['pricey'] = int(rests)

In [10]:
price_rests

{'inexp': 446, 'mod': 820, 'pricey': 107}

In [11]:
#calculate number of pages using number of resturants
pp_pages = dict()
for pp in price_rests:
    pages = (price_rests[pp]/30) + 1
    pp_pages[pp] = int(pages)

In [12]:
pp_pages

{'inexp': 15, 'mod': 28, 'pricey': 4}

## 2.0 Extract and Process all resturant names for each price point
### 2.1 Cheapest price point (Inexp)

In [13]:
#first we construct all URLs
inexp_page1

'https://www.yelp.com/search?find_desc=Restaurants&find_loc=western%20sydney&l=g%3A151.15106940214844%2C-33.67728783357405%2C150.7418286794922%2C-34.01944832216618&attrs=RestaurantsPriceRange2.1'

In [14]:
secondpage = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=western%20sydney&l=g%3A151.15106940214844%2C-33.67728783357405%2C150.7418286794922%2C-34.01944832216618&attrs=RestaurantsPriceRange2.1&start=30'

In [15]:
#After examining the URLs,'&start=30' is added to the end of the url for page 2 (30 is added for every page)
page2 = inexp_page1 + '&start=30'

In [16]:
#The last page url of any price point is therefore constructed by adding '&start=(MaxPages-1)*30'
page2

'https://www.yelp.com/search?find_desc=Restaurants&find_loc=western%20sydney&l=g%3A151.15106940214844%2C-33.67728783357405%2C150.7418286794922%2C-34.01944832216618&attrs=RestaurantsPriceRange2.1&start=30'

In [17]:
#lets now save this as resturant count, page 2 = 30, page 3 = 60....page 15 = 420
resturant_count = list()
#declare another list to keep all scrapped dirty names from the inexp price point
dirty_names = list()

In [18]:
for x in range(1,15):
    resturant_count.append(str((x*30)))

In [19]:
#Get resturant names from each page, start with page 1 then loop through the rest
response = requests.get(inexp_page1)
content = response.content
copycontent = str(content)
dirtynames = re.findall('target="" name=".{75}',copycontent)
dirty_names.append(dirtynames)
for count in resturant_count:
    new_page = inexp_page1 + '&start=' + count
    response = requests.get(new_page)
    content = response.content
    copycontent = str(content)
    dirtynames = list()
    dirtynames = re.findall('target="" name=".{75}',copycontent)
    dirty_names.append(dirtynames)

In [20]:
inexp_page1

'https://www.yelp.com/search?find_desc=Restaurants&find_loc=western%20sydney&l=g%3A151.15106940214844%2C-33.67728783357405%2C150.7418286794922%2C-34.01944832216618&attrs=RestaurantsPriceRange2.1'

In [21]:
dirty_names[0]

['target="" name="" rel="" role="link">Yelp</a></div></div><div class="lemon--div__373c0__1mb',
 'target="" name="" rel="" role="link"><span class="lemon--span__373c0__3997G text__373c0__2p',
 'target="" name="" rel="" role="link"><span class="lemon--span__373c0__3997G text__373c0__2p',
 'target="" name="" rel=""><img class="lemon--img__373c0__3GQUb photo-box-img__373c0__O0tbt" ',
 'target="" name="El Jannah Granville" rel="">El Jannah Granville</a></h3></div><div class="l',
 'target="" name="" rel="" role="link">Middle Eastern</a></span></span></span></div></div><di',
 'target="" name="" rel="">read more</a></span></p></div></div></div><div class="lemon--div__',
 'target="" name="" rel=""><img class="lemon--img__373c0__3GQUb photo-box-img__373c0__O0tbt" ',
 'target="" name="Aldhiaffah Al-Iraqi Restaurant" rel="">Aldhiaffah Al-Iraqi Restaurant</a></',
 'target="" name="" rel="" role="link">Middle Eastern</a></span></span></span></div></div><di',
 'target="" name="" rel="">read more</a>

In [22]:
#the captured strings need to be cleaned once more to extract only the resturant names:
clean_names = list()
for name in dirty_names:
    for string in name:
        cleanstring = re.findall('name="(.*?)"',string)
        clean_names.append(cleanstring)

In [23]:
clean_names[:10]

[[''],
 [''],
 [''],
 [''],
 ['El Jannah Granville'],
 [''],
 [''],
 [''],
 ['Aldhiaffah Al-Iraqi Restaurant'],
 ['']]

In [24]:
#we then need to convert the resturant names to strings
final_names = list()
for name in clean_names:
    name = str(name)
    name = name[2:]
    final_names.append(str(name))
    #print(name)

In [25]:
#now we need to clean the final names
clean_finalnames = list()
for name in final_names:
    if name[0] != "'":
        clean_finalnames.append(name[:len(name)-2])
        print(name)

El Jannah Granville']
Aldhiaffah Al-Iraqi Restaurant']
Frango']
Al Shami - Syrian &amp; Lebanese Cuisine']
Tan Viet Noodle House']
Hawa Charcoal Chicken']
Pho Ba Le Vietnamese Noodle Bar']
Darband Restaurant']
The Original Frango']
Gourmet Noodle House']
Awafi Charcoal Chicken']
Bau Truong Restaurant']
New Star Kebabs']
Pho Tau Bay']
Pho Lam Vietnamese Restaurant']
El Burger']
El Jannah']
Lao Village']
Chubby Buns Burger']
Anema E Core Pizzeria']
Mr Ping\\xe2\\x80\\x99s']
Plaza Noodle']
Rashays']
S Biber &amp; T Biber']
Knafeh']
Banh Cuon Kim Thanh']
Billu\\xe2\\x80\\x99s Express']
Tasty House Chinese Restaurant']
Simply Noodles']
Pho Gia Hoi']
Simply Noodles']
Jasmin Lebanese Restaurant']
Phu Quoc']
Sun\\xe2\\x80\\x99s Burmese Kitchen']
Plaza Noodle']
Pho Saigon']
Pho Ann']
Tasty House Chinese Restaurant']
S Biber &amp; T Biber']
Regents Park Food Centre']
J\\xe2\\x80\\x99s Kitchen Steak House']
Nando\\xe2\\x80\\x99s']
Billu\\xe2\\x80\\x99s Express']
The Usual Cafe']
Flamed &amp; Gril

In [26]:
len(clean_finalnames)

446

446 is the correct number of resturants, we just need to do 3 more things. Remove the - '] from the end of every name and replace \\xe2\\x80\\x99 with ' and replace amp& with nothing

In [27]:
#final phase of cleaning
inexp_names = list()
for name in clean_finalnames:
    _name = re.sub("]'",'',name)
    clean_name = re.sub("amp;",'',_name)
    final_name = re.sub(r'\\\\xe2\\\\x80\\\\x99',"'",clean_name)
    inexp_names.append(final_name)

In [29]:
inexp_names

['El Jannah Granville',
 'Aldhiaffah Al-Iraqi Restaurant',
 'Frango',
 'Al Shami - Syrian & Lebanese Cuisine',
 'Tan Viet Noodle House',
 'Hawa Charcoal Chicken',
 'Pho Ba Le Vietnamese Noodle Bar',
 'Darband Restaurant',
 'The Original Frango',
 'Gourmet Noodle House',
 'Awafi Charcoal Chicken',
 'Bau Truong Restaurant',
 'New Star Kebabs',
 'Pho Tau Bay',
 'Pho Lam Vietnamese Restaurant',
 'El Burger',
 'El Jannah',
 'Lao Village',
 'Chubby Buns Burger',
 'Anema E Core Pizzeria',
 "Mr Ping's",
 'Plaza Noodle',
 'Rashays',
 'S Biber & T Biber',
 'Knafeh',
 'Banh Cuon Kim Thanh',
 "Billu's Express",
 'Tasty House Chinese Restaurant',
 'Simply Noodles',
 'Pho Gia Hoi',
 'Simply Noodles',
 'Jasmin Lebanese Restaurant',
 'Phu Quoc',
 "Sun's Burmese Kitchen",
 'Plaza Noodle',
 'Pho Saigon',
 'Pho Ann',
 'Tasty House Chinese Restaurant',
 'S Biber & T Biber',
 'Regents Park Food Centre',
 "J's Kitchen Steak House",
 "Nando's",
 "Billu's Express",
 'The Usual Cafe',
 'Flamed & Grilled Seafoo